' Content-based recommender system in python '

summary:

In this system, I will show how to build a system that recommends movies similar to a specific movie. To achieve this, pairwise cosine similarity scores will be calculated for all movies based on their plot descriptions and I will recommend movies based on this similarity score threshold.

The next line is to import the pandas library, which is a good tool for analyzing and manipulating data. Pandas provides data structures like dataFrame which are good for dealing with tabular data and that's what I will be dealing with in this system.

In [1]:
# Import Pandas
import pandas as pd

In the next cell the movies metadata are loaded from the CSV file into the dataFrame for some processing. 
Then the first few rows of the data are previewed so I can understand the structure and content of the data.

In [2]:
# Load Movies Metadata
metadata = pd.read_csv('./archive/movies_metadata.csv', low_memory=False)
# Print the first three rows
metadata.head(3) 

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [3]:
metadata

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In the next cell, I reduced the size of the data by selecting it randomly to make it easier to work with, in addition to the small size of my memory.

In [4]:
metadata = metadata.sample(n=1000, random_state=1)  # Example: Subsetting to 1000 movies

#metadata = metadata.iloc[:1000]

Some rows of the data are previewed here so I can see the data I've randomly selected.

In [5]:
metadata

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
30160,False,"{'id': 256953, 'name': 'The Comics Collection'...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,43649,tt0159611,it,Le nuove comiche,A few funny little novels about different aspe...,...,1994-01-01,0.0,102.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,The New Comics,False,5.0,32.0
26084,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 80, ...",NaN,47212,tt0092751,en,China Girl,Teenage lovers Tony (Richard Panebianco) and T...,...,1987-09-25,1262091.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He's Italian. She's Chinese. Their gangs are s...,China Girl,False,6.6,8.0
37928,False,NaN,0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",NaN,133953,tt1841840,en,Snow Shark: Ancient Snow Beast,12 years ago during a scientific expedition 3 ...,...,2011-10-12,0.0,80.0,[],Released,Frozen in the ice for thousands of years... th...,Snow Shark: Ancient Snow Beast,False,4.0,2.0
929,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,43795,tt0034012,en,Penny Serenade,A light-hearted romantic drama starring Cary G...,...,1941-04-24,0.0,119.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"""Remember the tune they were singing the night...",Penny Serenade,False,6.4,23.0
7366,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,23981,tt0099857,en,Iron And Silk,Iron and Silk is a 1990 movie based on the epo...,...,1991-02-15,0.0,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"As a student in America, he searched for ancie...",Iron And Silk,False,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9679,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,61144,tt0102861,en,Scorchers,"Bayou La Teche, Louisiana sizzles as the Cajun...",...,1991-07-07,0.0,88.0,[],Released,NaN,Scorchers,False,4.8,3.0
6257,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,43361,tt0044000,it,Lo sceicco bianco,"The first two days of a marriage. Ivan, a punc...",...,1952-09-20,0.0,86.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,A Very Funny Picture by Federico Fellini,The White Sheik,False,7.2,29.0
10102,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,16932,tt0344604,fr,Après vous...,"Antoine is a maitre d' in a Paris brasserie, C...",...,2003-12-17,0.0,110.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,After You,False,6.9,14.0
9633,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,989,tt0032811,en,The Mortal Storm,The Roth family leads a quiet life in a small ...,...,1940-06-14,0.0,100.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Most Exciting Picture!,The Mortal Storm,False,7.1,15.0


Next part of the code accesses the 'overview' column of the metadata DataFrame. 
And then retrieves the first five rows. 

NOTE:
The “Overview” column contains summaries or descriptions of movie plots.

In [6]:
#Print plot overviews of the first 5 movies.
metadata['overview'].head()
# .head(5) == .head()

30160    A few funny little novels about different aspe...
26084    Teenage lovers Tony (Richard Panebianco) and T...
37928    12 years ago during a scientific expedition 3 ...
929      A light-hearted romantic drama starring Cary G...
7366     Iron and Silk is a 1990 movie based on the epo...
Name: overview, dtype: object

Here I will convert a set of raw documents into a matrix of TF-IDF features using TfidfVectorizer which I called from sklearn.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

The purpose of next steps in the next cell is to preprocess the movie overviews and convert them into a numerical representation that can be used for computing similarities between movies based on their descriptions. 
(compute TF-IDF vectors for each overview)

To clarify more:

* TF-IDF Vectorizer: Converts text data into a numerical matrix of TF-IDF features by initializes a TfidfVectorizer object.
Note:
By stop_words='english' I can remove common English stop words (like 'and','the', 'a'..) from the text before vectorizing, becuase they don't give any useful information.

* Handling NaNs: This step is to ensure that there are no missing values that disrupt text processing. (replaces any missing values (NaNs) in the Overview column with empty strings)

* fit_transform does two things: it learns the vocabulary and inverse document frequency (IDF) from the 'overview' data, and it transforms the data into a matrix of TF-IDF features.

* The result, tfidf_matrix, is a sparse matrix where each row corresponds to a movie and each column corresponds to a term from the overviews, with the value representing the TF-IDF score for that term in that document.

* Matrix Shape: This line outputs the shape of the TF-IDF matrix.

In [8]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(1000, 9829)

From the above output, it can be seen that 9829 different vocabularies or words in the dataset have 1000 movies.

With this matrix in hand, I can now compute a similarity score. I will be use the cosine similarity to calculate a numeric quantity that denotes the similarity between two movies.

This line retrieves the feature names (terms) corresponding to the TF-IDF matrix columns and prints a subset of them (specifically the 5000th to 5009th terms).

In [9]:
# Array mapping from feature integer indices to feature name using the updated method.
feature_names = tfidf.get_feature_names_out()
print(feature_names[5000:5010])

['lagging' 'lago' 'laid' 'lainie' 'lair' 'lake' 'lakes' 'lakewood'
 'lalonde' 'lamar']


The linear_kernel function computes the dot product between two matrices. When applied to TF-IDF vectors, the linear kernel is equivalent to cosine similarity, which is a measure of similarity between two non-zero vectors.

(computes the cosine similarity between all pairs of movie overviews. Hence, each movie will be a 1x1000 column vector where each column will be a similarity score with each movie.)
_________
The purpose of these steps is to calculate the similarity between movies based on their descriptions. The cosine similarity matrix will be used to identify movies that are similar to each other, which is crucial for making recommendations in my content-based recommender system.

In [10]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

The next code gives the dimensions of the cosine similarity matrix. The shape of this matrix tells the following:

* Number of Movies: The number of rows (and columns) in the matrix corresponds to the number of movies in the dataset.

* Pairwise Similarities: Since it's a square matrix, each entry [i,j] corresponds to the similarity between the i-th and j-th movies.

In [11]:
cosine_sim.shape

(1000, 1000)

In [12]:
cosine_sim

array([[1.        , 0.058119  , 0.        , ..., 0.01302362, 0.02441997,
        0.        ],
       [0.058119  , 1.        , 0.        , ..., 0.        , 0.03493912,
        0.01748445],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01302362, 0.        , 0.        , ..., 1.        , 0.00672325,
        0.00577064],
       [0.02441997, 0.03493912, 0.        , ..., 0.00672325, 1.        ,
        0.        ],
       [0.        , 0.01748445, 0.        , ..., 0.00577064, 0.        ,
        1.        ]])

Accesses the second row of the cosine similarity matrix.

* The purpose of accessing cosine_sim[1] is to retrieve the similarity scores for the second movie in relation to all other movies.

In [13]:
cosine_sim[1]

array([0.058119  , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01177505, 0.01415235, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00918726, 0.        , 0.        , 0.        , 0.04522296,
       0.        , 0.05425354, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.0420487 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.0151142 ,
       0.        , 0.        , 0.00996669, 0.01887813, 0.07747762,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02982799,
       0.        , 0.        , 0.02627957, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.04334

Here a reverse map of indicators and movie titles will be generated.

This line creates a Pandas Series indices where the index is the movie title and the value is the corresponding index in the metadata DataFrame.

The purpose of constructing this reverse map (indices) is to easily look up the index of a movie based on its title. 
This is particularly useful when we want to make recommendations based on movie titles identified during the recommendation process.

* I'll do this so it helps me after a while when I define a function that takes the movie title as input and outputs a list of the 10 most similar movies.

In [14]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

Next code accesses the first 10 entries (rows) of the indices Pandas Series. Where movie titles are used as the index, and the corresponding values are the indices (row numbers) in the metadata DataFrame.

The purpose of accessing for them is to examine the first 10 entries in the indices Series. This useful for understanding how the mapping between movie titles and their indices is structured.

In [23]:
indices[:10]

title
The New Comics                                           30160
China Girl                                               26084
Snow Shark: Ancient Snow Beast                           37928
Penny Serenade                                             929
Iron And Silk                                             7366
Lassie                                                   11227
Michael Jackson's Journey from Motown to Off the Wall    35973
Whitney                                                  41155
Razend                                                   41066
Hungry Hill                                              25226
dtype: int64

The purpose of the get_recommendations -next- function is to provide a list of movie recommendations based on the similarity of their content. It uses a content-based filtering approach by comparing the textual descriptions (overviews) of movies to find those that are most similar to a given movie.

This function has several tasks:

* Input: Movie title.
* Output: List of top 10 most similar movie titles.
* Process:
   1.  Get the index of the given movie.
   2.  Compute similarity scores between the given movie and all other movies.
   3.  Sort the similarity scores.
   4.  Select the top 10 most similar movies (excluding the movie itself).
   5.  Return the titles of these similar movies.

In [16]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]


In the next cell I will be find movies similar to "Penny Serenade"

In [17]:
get_recommendations('Penny Serenade')

21624         Aurora Borealis
15000             Yuki & Nina
4014              North Shore
19403             Kinyarwanda
5856        Nicholas Nickleby
14654             The V.I.P.s
22143    August: Osage County
4143                   Eureka
39672               U Be Dead
9633         The Mortal Storm
Name: title, dtype: object

In the next cell I will be find movies similar to "Cobb"

In [24]:
get_recommendations('Cobb')

34767    Don't Open Till Christmas
35472     When Santa Fell to Earth
2996                   End of Days
27484         Santa's Pocket Watch
33704                    The Tower
271                     Mixed Nuts
19212            A Christmas Carol
35205     Rudolph's Shiny New Year
3203              The Waiting Game
5873                     WiseGirls
Name: title, dtype: object

While the system did a good job of finding movies with similar plot descriptions, the quality of recommendations is not good enough, so I want to improve the system by doing the following.
I will create a recommendation system based on the following metadata: top 3 actors, director, related genres, and movie plot keywords.

* Note: The keywords, cast, and crew data are not available in current dataset, so the first step would be to load and merge them into main DataFrame metadata.

Purpose of the Code:
* Data Loading: Loads additional datasets that contain complementary information such as cast, crew, and movie keywords.
* Data Cleaning: Removes specific rows from the metadata that having problems
* Data Preparation: Ensures that all 'id' columns across different DataFrames (metadata, credits, keywords) are of the same integer type, facilitating seamless merging.
* Data Merging: Integrates the information from credits (cast and crew) and keywords (genres and tags) into the main metadata DataFrame. 

After all of these, the metadata DataFrame will contain enriched information from credits and keywords, making it suitable for various analytical tasks.

In [25]:
# Load keywords and credits
credits = pd.read_csv('./archive/credits.csv')
keywords = pd.read_csv('./archive/keywords.csv')

# Remove rows with bad IDs.
#metadata = metadata.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

Printing the first two movies from metadata provides an initial glimpse into the combined dataset, confirming that the merging process was successful. 

In [26]:
# Print the first two movies of your newly merged metadata
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 256953, 'name': 'The Comics Collection'...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,43649,tt0159611,it,Le nuove comiche,A few funny little novels about different aspe...,...,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,The New Comics,False,5.0,32.0,"[{'cast_id': 5, 'character': 'Paolo', 'credit_...","[{'credit_id': '52fe4657c3a36847f80f9b29', 'de...",[]
1,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 80, ...",NaN,47212,tt0092751,en,China Girl,Teenage lovers Tony (Richard Panebianco) and T...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He's Italian. She's Chinese. Their gangs are s...,China Girl,False,6.6,8.0,"[{'cast_id': 2, 'character': 'Alby', 'credit_i...","[{'credit_id': '52fe4725c3a36847f8125cd5', 'de...","[{'id': 497, 'name': 'shakespeare'}, {'id': 54..."


The Purpose of the next code:

* Parsing Stringified Features: By applying literal_eval, I will convert the stringified representations back into their original Python objects, making them usable for analysis or manipulation.

* Data Preparation: This preprocessing step ensures that features like cast, crew, keywords, and genres are properly converted into their appropriate data types for subsequent analysis tasks.
___________________
Next step is important step in converting stringified representations of Python objects into their native Python types (list, dict, ) 

We do this to extract the three most important actors, the director and the keywords associated with that movie from new features cast, crew, and keywords.

In [27]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

I'll import the NumPy package to get access to NaN constant. Next, I will use it to write the get_director function

In [28]:
# Import Numpy
import numpy as np

Now I will write functions that will help me to extract the required information from each feature.

The function get_director is designed to extract the name of the director from a list of dictionaries (x).

( iterates through a list of dictionaries (x), checks for the presence of a dictionary with 'job' equal to 'Director', and returns the corresponding director's name)

 It handles cases where director information might be missing by returning np.nan

In [29]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


The purpose of the next function:

* Data Extraction: The get_list function is designed to extract names from a list of dictionaries (x), where each dictionary represents an entity (e.g., cast members, keywords).
* Data Cleaning: It ensures that only valid lists are processed, and it handles cases where data might be missing by returning an empty list.
* Limiting Output: It limits the output to a maximum of three names per list to manage data size.

In [30]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []


In the next code we will find:
* Extracting the Director: The  get_director function is applied to the crew column to create a director column with the director's name.

* Transforming Lists of Dictionaries: The get_list function is applied to the cast, keywords, and genres columns to create lists of names for each movie.

These transformations prepare the features for similarity computation in a content-based recommender system.

In [31]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)


This part selects specific columns from the metadata DataFrame. And retrieves the first 3 rows from the DataFrame.

* Each of them for movies.
* The purpose of this code snippet is to display the newly engineered features. 

In [32]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,The New Comics,"[Paolo Villaggio, Renato Pozzetto, Ramona Bade...",Neri Parenti,[],[Comedy]
1,China Girl,"[James Russo, Richard Panebianco, David Caruso]",Abel Ferrara,"[shakespeare, street gang, chinatown]","[Romance, Crime, Drama]"
2,Snow Shark: Ancient Snow Beast,"[Sam Qualiana, Michael O'Hear, Jackey Hall]",Sam Qualiana,"[monster, shark attack]","[Science Fiction, Horror]"


The next function is to convert the names and instances of keywords to lowercase and remove all spaces between them.
This is important for vectorization.

In [33]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


The main purpose of the next code is to clean textual data within specific columns of the metadata DataFrame.

By applying the clean_data function to each feature, it ensures that all text data within these columns is converted to lower case and stripped of unnecessary spaces. 

In [34]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)


The create_soup function creates a consolidated textual representation ('soup') of multiple features ('keywords', 'cast', 'director', 'genres') associated with a movie. By joining these features into a single string, and then I can feed this soup into a word vector model.

In [35]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


This line applies the create_soup function row-wise across the metadata DataFrame. That specifies by The axis=1 parameter.

Assigns the result  to a new column 'soup' in the metadata DataFrame.
Each row in 'soup' will contain the consolidated textual representation ('soup') of multiple features ('keywords', 'cast', 'director', 'genres') 

In [36]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [37]:
metadata[['soup']].head(2)

,soup
0,paolovillaggio renatopozzetto ramonabadescu n...
1,shakespeare streetgang chinatown jamesrusso ri...


The next steps are the same as what I did in the prevous with my plot description based recommender. One key difference is that I use the CountVectorizer() instead of TF-IDF. This is because I don't want to down-weight the actor/director's presence if he or she has acted or directed in relatively more movies. It doesn't make much intuitive sense to down-weight them in this context.

The major difference between CountVectorizer() and TF-IDF is the inverse document frequency (IDF) component which is present in later and not in the former.

* Importing CountVectorizer: To transform text data into a matrix of token counts.
* Creating CountVectorizer Object: Initializing with English stop words removed to clean the text data.
* Fitting and Transforming Data: Converting the 'soup' feature into a count matrix, which is a crucial step for subsequent similarity calculations.

In [38]:
#last
# Import CountVectorizer and create the count matrix

from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])


In [39]:
count_matrix.shape

(1032, 4490)

From the above output, we can see that there are 4490 vocabularies in the metadata that I fed to it. And then I will use the cosine_similarity to measure the distance between the embeddings.

* computes the Cosine Similarity matrix based on the count_matrix created using CountVectorizer.
* The main purpose of computing cosine similarity is to measure the similarity between movies based on their textual features. 
* Higher cosine similarity values indicate greater similarity.

In [40]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In the next code two important things are done:
1. Resetting DataFrame Index 
    
2. Constructing Reverse Mapping: Constructs a reverse mapping where the movie titles ('title' column) are set as the index of a pandas Series (indices). The values of this Series are the corresponding indices (row numbers) of the metadata DataFrame.

In [41]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

In [42]:
metadata

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,tagline,title,video,vote_average,vote_count,cast,crew,keywords,director,soup
0,0,False,"{'id': 256953, 'name': 'The Comics Collection'...",0,[comedy],NaN,43649,tt0159611,it,Le nuove comiche,...,NaN,The New Comics,False,5.0,32.0,"[paolovillaggio, renatopozzetto, ramonabadescu]","[{'credit_id': '52fe4657c3a36847f80f9b29', 'de...",[],neriparenti,paolovillaggio renatopozzetto ramonabadescu n...
1,1,False,NaN,0,"[romance, crime, drama]",NaN,47212,tt0092751,en,China Girl,...,He's Italian. She's Chinese. Their gangs are s...,China Girl,False,6.6,8.0,"[jamesrusso, richardpanebianco, davidcaruso]","[{'credit_id': '52fe4725c3a36847f8125cd5', 'de...","[shakespeare, streetgang, chinatown]",abelferrara,shakespeare streetgang chinatown jamesrusso ri...
2,2,False,NaN,0,"[sciencefiction, horror]",NaN,133953,tt1841840,en,Snow Shark: Ancient Snow Beast,...,Frozen in the ice for thousands of years... th...,Snow Shark: Ancient Snow Beast,False,4.0,2.0,"[samqualiana, michaelo'hear, jackeyhall]","[{'credit_id': '542633270e0a2659f3000a33', 'de...","[monster, sharkattack]",samqualiana,monster sharkattack samqualiana michaelo'hear ...
3,3,False,NaN,0,"[drama, romance]",NaN,43795,tt0034012,en,Penny Serenade,...,"""Remember the tune they were singing the night...",Penny Serenade,False,6.4,23.0,"[irenedunne, carygrant, beulahbondi]","[{'credit_id': '52fe465fc3a36847f80fb6cb', 'de...","[honeymoon, sterility, daughter]",georgestevens,honeymoon sterility daughter irenedunne carygr...
4,4,False,NaN,0,"[drama, comedy]",NaN,23981,tt0099857,en,Iron And Silk,...,"As a student in America, he searched for ancie...",Iron And Silk,False,6.0,1.0,"[marksalzman, panqingfu, vivianwu]","[{'credit_id': '5998ab5fc3a3680f4e00b626', 'de...","[martialarts, kungfu, basedonnovel]",shirleysun,martialarts kungfu basedonnovel marksalzman pa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,1027,False,NaN,0,"[comedy, drama, romance]",NaN,61144,tt0102861,en,Scorchers,...,NaN,Scorchers,False,4.8,3.0,"[fayedunaway, denholmelliott, jamesearljones]","[{'credit_id': '52fe4647c3a368484e088037', 'de...",[independentfilm],davidbeaird,independentfilm fayedunaway denholmelliott jam...
1028,1028,False,NaN,0,"[romance, comedy, drama]",NaN,43361,tt0044000,it,Lo sceicco bianco,...,A Very Funny Picture by Federico Fellini,The White Sheik,False,7.2,29.0,"[albertosordi, leopoldotrieste, giuliettamasina]","[{'credit_id': '52fe4642c3a36847f80f4edb', 'de...","[runaway, honeymoon, camel]",federicofellini,runaway honeymoon camel albertosordi leopoldot...
1029,1029,False,NaN,0,"[comedy, romance]",NaN,16932,tt0344604,fr,Après vous...,...,NaN,After You,False,6.9,14.0,"[danielauteuil, sandrinekiberlain, marilynecanto]","[{'credit_id': '52fe46f89251416c75089a43', 'de...",[love],pierresalvadori,love danielauteuil sandrinekiberlain marilynec...
1030,1030,False,NaN,0,[drama],NaN,989,tt0032811,en,The Mortal Storm,...,The Most Exciting Picture!,The Mortal Storm,False,7.1,15.0,"[margaretsullavan, jamesstewart, robertyoung]","[{'credit_id': '52fe4297c3a36847f802a8d7', 'de...","[germany, austria, professor]",frankborzage,germany austria professor margaretsullavan jam...


Now I reuse get_recommendations() function by passing in the new cosine_sim2 matrix as a second argument.

In [48]:
get_recommendations('Cobb', cosine_sim2)

91                          Blue Chips
237    Mr Hockey The Gordie Howe Story
887            The Game of Their Lives
199          When the Game Stands Tall
247                           Wojaczek
58                             Madison
166                           The King
357                       Hope Springs
529                         100 Meters
608             Angels in the Outfield
Name: title, dtype: object

In [49]:
get_recommendations('The New Comics', cosine_sim2)

42                                        Bill Cosby: 49
427                                      Mina Tannenbaum
500                                     Bienvenue à bord
666    Stewart Lee: If You Prefer a Milder Comedian, ...
732                                      Questi fantasmi
764                                  Tom, Dick And Harry
865                                     Kiss and Make-Up
832                                     The Waiting Game
900                          Cheech & Chong's Next Movie
300                                      Noi uomini duri
Name: title, dtype: object

Here the recommender has been successful in capturing more information due to more metadata and has given better recommendations. 

In the next cell, I will summarize what I learned from this code, I will describe the algorithm I used, an idea to extend this work to a collaborative filtering approach, and the last thing is the conclusion.

In the first, for what I learned, I learn how I can use cosine similarity  to compute the pairwise similarity between movies based on the TF-IDF and count matrices, Sorting similarity scores to find and return the most similar movies and Data Preprocessing like Handling Missing Values also Data Cleaning.

As for the algorithm that I used, this code implements the content-based recommendation system using the following steps:

1. TF-IDF Vectorization
2. Cosine Similarity
3. Count Vectorization and Feature Aggregation
4. Recommendation Function

Idea to Extend This Work for Collaborative Filtering:
The system can be hybrid, I mean taking the advantages from each of them and combining them into one system.

As a conclusion:

This code demonstrates a content-based recommendation system that uses movie metadata to recommend similar movies. By leveraging text vectorization and cosine similarity techniques, it effectively identifies movies with similar content. 

Expanding this system to include collaborative filtering would enhance its capabilities by leveraging user interaction data, thus providing more personalized recommendations. A hybrid approach, which combines between them can leverage the strengths of both and provide robust recommendations .